In [1]:
import math
import time
import wandb
import rpnn
import pprint
#import sys
#!{sys.executable} -m pip install rpnn

In [2]:
from IPython.display import display, clear_output

import gym
env = gym.make('CartPole-v1')

ts = []
worst_average_timesteps = 1000
best_average_timesteps = 0

In [3]:
def log_run(t):
    global ts
    global worst_average_timesteps
    global best_average_timesteps
    
    random_policy_average_timesteps = 24
    oppose_tilt_policy_average_timesteps = 40
    good_average_timesteps = 160
    
    ts.append(t+1)
    if(len(ts) >= 50):
        clear_output(wait=True)
        perf = sum(ts)/len(ts)

        new_best = perf > best_average_timesteps
        if(new_best):
            best_average_timesteps = perf

        new_worst = perf < worst_average_timesteps
        if(new_worst):
            worst_average_timesteps = perf

        success_now = perf > random_policy_average_timesteps
        big_success = perf > oppose_tilt_policy_average_timesteps
        huge_success = perf > good_average_timesteps

        RED='\033[91m'
        GREEN='\033[92m'
        DEFAULT='\033[0m'
        BLUE='\033[94m'
        BOLD='\033[1m'
        ORANGE='\033[93m'

        print("Average timesteps: {}{}{} {}Best {}{} {}Worst {}{}".format(
            BLUE+BOLD if huge_success else GREEN+BOLD if big_success else GREEN if success_now else RED, perf, DEFAULT,
            GREEN if new_best else DEFAULT, best_average_timesteps, DEFAULT,
            RED if new_worst else DEFAULT, worst_average_timesteps, DEFAULT))

        ts = []

In [4]:
def reset_environment():
    observation = env.reset()
    reward = 0
    done = False
    return (observation, reward, done)

observation, reward, done = reset_environment()

In [ ]:
enable_rendering = True

while True:
    for t in range(1000):
        if enable_rendering: env.render()
        
        action = rpnn.cartpole(observation.tolist(), reward, done, False)

        observation, reward, done, info = env.step(action)

        if done:
            log_run(t)
            observation, reward, done = reset_environment()
            break

Average timesteps: 128.18 Best 306.82 Worst 17.96


In [ ]:
env.close()